# OPTIMIZACIÓN BAYESIANA

In [1]:
import pandas as pd
import numpy as np

import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import hp


import os
import lightgbm as lgb

In [2]:
#CARGO DATASET
os.chdir("C:/Users/vyago/Desktop/Yago/Competencia/ypf")  # Directorio actual
train = pd.read_csv("../Dataset/dataset_train.csv")


In [3]:
train = train.select_dtypes("number")
y_train = train["delta_WHP"]
X_train = train[train.columns.drop(["delta_WHP","ID_FILA"])]#,"ID_EVENTO"])]

In [4]:

MAX_EVALS = 500
N_FOLDS = 5

def objective(params, n_folds = N_FOLDS):
   """Función objetivo para la Optimización de hiperparametros del Gradient Boosting Machine"""
   
   # Llevar el conteo de iteraciones
   global ITERATION
   ITERATION += 1
   
   # Recupera el subsample si se encuentra, en caso contrario se asigna 1.0
   #subsample = params['boosting_type'].get('subsample', 1.0)
   
   # Extrae el boosting type
   params['boosting_type'] = 'gbdt'
   params['subsample'] = 1.0
  
   # Se asegura que los parametros que tienen que ser enteros sean enteros
   for parameter_name in ['num_leaves', 
                          'min_data_in_leaf']:
       params[parameter_name] = int(params[parameter_name])
   start = timer()
   params["max_bin"] = 128
   params['objective'] = 'regression'
   # realiza n_folds de cross validation
   cv_results = lgb.cv(params, train_set, num_boost_round = 9999999,
                       nfold = n_folds, early_stopping_rounds = int(50+5/params["learning_rate"]) , 
                       metrics = 'rmse', seed = 50,stratified=False)
   
   run_time = timer() - start
   
   # Extrae el mejor score
   best_score = np.max(cv_results['rmse'])
   
   # El loss se debe minimizar
   loss = 1 - best_score
   
   # Impulsando las iteraciones que arrojaron el mayor score en CV
   n_estimators = int(np.argmax(cv_results['rmse']) + 1)
   
   # Escribe sobre el archivo CSV ('a' significa append)
   of_connection = open(out_file, 'a')
   writer = csv.writer(of_connection)
   writer.writerow([loss, params, ITERATION, n_estimators, 
                   run_time])
   
   # Dictionary con informacion para la evaluación
   return {'loss': loss, 'params': params, 'iteration': ITERATION,
           'estimators': n_estimators, 'train_time': run_time, 
           'status': STATUS_OK}

In [15]:
#ESPACIO DE BUSQUEDA EJEMPLO
## LO ANULO, ME INTERESA EXPLORAR OTROS
"""
space = {
'class_weight': hp.choice('class_weight', [None, 'balanced']),
'boosting_type': hp.choice('boosting_type', [
{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)},
{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
{'boosting_type': 'goss', 'subsample': 1.0}]),
'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
'learning_rate': hp.loguniform('learning_rate', np.log(0.01),np.log(0.2)),
'subsample_for_bin': hp.quniform('subsample_for_bin', 20000,300000),
'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)}"""

"\nspace = {\n'class_weight': hp.choice('class_weight', [None, 'balanced']),\n'boosting_type': hp.choice('boosting_type', [\n{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)},\n{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},\n{'boosting_type': 'goss', 'subsample': 1.0}]),\n'num_leaves': hp.quniform('num_leaves', 30, 150, 1),\n'learning_rate': hp.loguniform('learning_rate', np.log(0.01),np.log(0.2)),\n'subsample_for_bin': hp.quniform('subsample_for_bin', 20000,300000),\n'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),\n'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),\n'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),\n'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)}"

In [5]:
space = {
'num_leaves': hp.quniform('num_leaves', 100, 2000, 1),
'learning_rate': hp.loguniform('learning_rate', np.log(0.01),np.log(0.2)),
'min_data_in_leaf': hp.quniform('min_data_in_leaf', 200, 4000, 5),
'feature_fraction': hp.uniform('feature_fraction', 0.1, 1.0)
}



In [7]:

# Algoritmo de optimización
tpe_algorithm = tpe.suggest

# Lleva el registro de los resultados
bayes_trials = Trials()

# archivo para guardar los primeros resultados
out_file = "../Exp/HT.csv"
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# escribe la cabecera de los archivos
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

In [27]:

# Variable Global
global  ITERATION
ITERATION = 0
MAX_EVALS = 300

# Crea un dataset lgb
train_set = lgb.Dataset(X_train, label = y_train)

# Corre la optimización
best = fmin(fn = objective, space = space, algo = tpe.suggest,
            max_evals = MAX_EVALS, trials = bayes_trials, 
            rstate=np.random.default_rng(50))

  0%|          | 0/300 [00:00<?, ?trial/s, best loss=?]

c:\Users\vyago\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

job exception: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.



  0%|          | 0/300 [00:00<?, ?trial/s, best loss=?]


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.